In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [2]:
# functions
# temporary market impact function
def h(n, tau, epsilon, eta):
    return epsilon*np.sign(n) + eta*(n/tau)

# permanent market impact function
def g(n, tau, alpha):
    """
    in the article it's gamma but to follow the ACDP equations given to us, 
    this gamma was replaced by alpha
    """
    return alpha*(n/tau) 
# change of variable: new equation
def H(q, n, gamma, sigma, tau,alpha,epsilon, eta):
    return (0.5*(gamma**2)*((q-n)**2)*(sigma**2)*tau + 
            gamma*n*h(n,tau,epsilon,eta) + 
            gamma*(q-n)*g(n,tau,alpha)*tau)


In [3]:
def efficient_frontier(X_total, V_total, S0, r, nb_T, tau, alpha, eta, epsilon, sigma, type_func):
    
    def optimization(type_func):
        """
        get Optimal trajectories
        """
        x_init  = np.ones((nb_T,1)) * X_total / nb_T # constant sell initialization
        bounds = [(0.0, X_total) for x in range(len(x_init))]
        constraints = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - X_total})


        optim = opt.minimize(objective_func(type_func), x_init, method='SLSQP', bounds=bounds, constraints=constraints) 
        q_opt = np.array(optim.x)

        return q_opt
    
    def Exp_IS(alpha, epsilon, eta, tau, q_opt, X_total):
        """
        IS Expectation
        """
        # with the linear costs model we have:
        exp_IS = 0.5*alpha*(X_total**2) + epsilon*np.sum(q_opt) + (eta - 0.5*alpha*tau)/tau * np.sum(q_opt**2)

        return exp_IS

    def Var_IS(tau, sigma, q_opt):
        """
        IS variance
        """
        
        return (tau* (sigma **2) * np.sum(np.cumsum(X_total - q_opt)**2))
        
    
    def Exp_VWAP(X_total, nb_T, V_total, S0, q_opt, alpha, epsilon, eta, tau ):
        """
        VWAP Expectation
        """

        v = np.ones((nb_T,1)) * V_total / nb_T
        nbr_opt = len(q_opt)

        exp_vwap = 0
        for i in range(nbr_opt):
            sum_g_i = 0
            for j in range(i):
                sum_g_i += g(q_opt[j], tau, alpha)
            exp_vwap += (X_total*v[i]/V_total - q_opt[i])*(S0-sum_g_i*tau) - X_total*tau*v[i]*g(q_opt[i], tau, alpha)/V_total  + q_opt[i]*h(q_opt[i], tau, epsilon, eta)
        return exp_vwap

    def Var_VWAP(X_total, nb_T, S0, q_opt, tau, V_total):
        """
        VWAP variance
        """

        nbr_opt = len(q_opt)
        v = np.ones((nb_T,1)) * V_total / nb_T
        var_vwap = 0
        for i in range(nbr_opt):
            sum_volum = 0
            for j in range(i,nbr_opt):
                sum_volum += v[j]
            #var_vwap = X_total - np.sum(q_opt[0:i]) 
            var_vwap+=(X_total/V_total * sum_volum - q_opt[i])**2 *tau*sigma**2


        return var_vwap

    def Exp_TWAP(X_total, nb_T, S0, q_opt, alpha, epsilon, eta, tau):
        """
        TWAP Expectation
        """
        
        T_total = tau*nb_T
        exp_twap = 0
        nbr_opt = len(q_opt)
        for i in range(nbr_opt):
            sum_g_i=0
            for j in range(i):
                sum_g_i += g(q_opt[j], tau, alpha)
            exp_twap += (X_total*tau/T_total - q_opt[i])*(S0-tau*sum_g_i) - (X_total/T_total) * (tau**2)*g(q_opt[i], tau, alpha) + q_opt[i]*h(q_opt[i], tau, epsilon, eta)
        return exp_twap

    def Var_TWAP( q_opt, tau, sigma, X_total):
        """
        TWAP variance
        """
        var_twap=0
        nbr_opt = len(q_opt)
        T_total = nbr_opt * tau
        for i in range(nbr_opt):
            sum_tau = 0
            for j in range(i,nbr_opt):
                sum_tau+=tau
            x = X_total - np.sum(q_opt[0:i])
            var_twap+=tau*(sigma**2)*((X_total/T_total * sum_tau-x)**2)
        return var_twap

    
    def objective_func(type_func):
        if type_func == 'VWAP':
            return objective_VWAP
        elif type_func == 'TWAP':
            return objective_TWAP
        elif type_func == 'IS':
            return objective_IS
        
    def objective_IS(q_opt):
        """
        Objective function for the IS
        """
        func = Exp_IS(alpha, epsilon, eta, tau, q_opt, X_total) + r * Var_IS(tau, sigma, q_opt)
        return func
    
    def objective_VWAP(q_opt):
        """
        Objective function for the VWAP
        """
        func = Exp_VWAP(X_total,nb_T, V_total, S0, q_opt, alpha, epsilon, eta, tau ) + r * Var_VWAP(X_total, nb_T, S0, q_opt, tau, V_total)
        return func
    
    def objective_TWAP(q_opt):
        """
        Objective function for the TWAP
        """
        func = Exp_TWAP(X_total, nb_T, S0, q_opt, alpha, epsilon, eta, tau) + r * Var_TWAP(q_opt,tau,sigma, X_total)
        return func
    
    def reorder(q_opt):
        """
        Reorder the inventory liquidation and get the increasing liquidation amount
        """
        return  np.cumsum(q_opt[::-1])[::-1]
    
    q_opt = optimization(type_func)
    list_opt = reorder(q_opt)
    
    if type_func == 'VWAP':
        res = list_opt, Exp_VWAP(X_total,nb_T, V_total, S0, q_opt, alpha,
                                 epsilon, eta, tau), Var_VWAP(X_total, nb_T, S0, 
                                                              q_opt, tau, V_total)
    
    elif type_func == 'TWAP':
        res = list_opt, Exp_TWAP(X_total, nb_T, S0, q_opt, alpha, 
                                 epsilon, eta, tau), Var_TWAP( q_opt, tau, sigma, X_total)
    
    elif type_func == 'IS':
        res = list_opt, Exp_IS(r, epsilon, eta, tau, q_opt, 
                               X_total), Var_IS(tau, sigma, q_opt)
    

    return res    
        

In [4]:
def compare_risk_av(risks, X_total,V_total, S0, nb_T, 
                                 tau, alpha, eta, epsilon, sigma,bench_type):
    
    """
    Plotting various optimal trajectories with different risk aversion
    """
    
    q_opts = []
    # in case they are not sent in an increasing order: sort them
    risks.sort()
    for r in risks:
        temp = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon,sigma,bench_type)
        
        q_opts.append(temp[0])


    # Plotting
    plt.figure()
    plt.plot(q_opts[0], label = 'risk aversion: ' + str(risks[0]))
    plt.plot(q_opts[1], label = 'risk aversion: ' + str(risks[1]))
    plt.plot(q_opts[2], label = 'risk aversion: ' + str(risks[2]))
    plt.plot(q_opts[3], label = 'risk aversion: ' + str(risks[3]))
    plt.plot(q_opts[4], label = 'risk aversion: ' + str(risks[4]))
    plt.grid(True)
    plt.title('Optimal trajectories with '+ bench_type)
    plt.xlabel('Trading periods')
    plt.ylabel('Remaining Inventory')
    plt.legend()
    plt.show()



# IS

In [ ]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'IS'
EF_IS = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon, sigma, type_func)
plt.figure()
plt.plot(EF_IS[0], 'b')
plt.grid(True)
plt.xlabel('Trading periods')
plt.ylabel('Remaining Inventory')
plt.title("Liquidation path for " + type_func +" and risk aversion of: " +str(r))
plt.show()


In [ ]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'IS'
risks = [-1e-8, -1e-9, 0.0, 1e-07, 1e-08]

compare_risk_av(risks, X_total,V_total, S0, nb_T, 
                                 tau, alpha, eta, epsilon,sigma, type_func)

In [ ]:
a, b = 1e-07, 1e-08
risks = np.linspace(a, b, 300)
type_func = 'IS'
x = []
y = []
q_trajectories =[]
for r in risks:
    temp = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon,sigma, type_func)
    q_trajectories.append(temp[0])
    x.append(temp[2])
    y.append(temp[1])
    
plt.figure(figsize=(12,6))
plt.scatter(x, y)
plt.grid(True)
plt.xlabel('Variance V(x)')
plt.ylabel('Expectation E(x)')
plt.title('Efficient frontier for '+ type_func)
plt.show()

# TWAP

In [ ]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'TWAP'
EF_TWAP = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon, sigma, type_func)
plt.figure()
plt.plot(EF_TWAP[0], 'b')
plt.grid(True)
plt.xlabel('Trading periods')
plt.ylabel('Remaining Inventory')
plt.title("Liquidation path for " + type_func +" and risk aversion of: " +str(r))
plt.show()


In [ ]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'TWAP'
risks = [-1e-7, -1e-5, 0.0, 1e-07, 1e-06]

compare_risk_av(risks, X_total,V_total, S0, nb_T, 
                                 tau, alpha, eta, epsilon, sigma, type_func)


In [ ]:
a, b = 1e-06, 1e-07
risks = np.linspace(a, b, 300)
type_func = 'TWAP'
x = []
y = []
q_trajectories =[]
for r in risks:
    temp = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon,sigma, type_func)
    q_trajectories.append(temp[0])
    x.append(temp[2])
    y.append(temp[1])
    
plt.figure(figsize=(12,6))
plt.scatter(x, y)
plt.grid(True)
plt.xlabel('Variance V(x)')
plt.ylabel('Expectation E(x)')
plt.title('Efficient frontier for '+ type_func)
plt.show()

# VWAP

In [ ]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'VWAP'
EF_VWAP = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon, sigma, type_func)
plt.figure()
plt.plot(EF_VWAP[0], 'b')
plt.grid(True)
plt.xlabel('Trading periods')
plt.ylabel('Remaining Inventory')
plt.title("Liquidation path for " + type_func +" and risk aversion of: " +str(r))
plt.show()


In [14]:
nb_T = 60
X_total= 50000
V_total = 500000
alpha = 2.5*10**(-7)
sigma = 0.3
tau= 0.5
r = 2.0*10**(-7)
epsilon = 0.0625
eta= 2.5*10**(-6)
S0 =100
type_func = 'VWAP'
risks = [-1e-4, -1e-5, 0.0, 1e-04, 1e-05]

compare_risk_av(risks, X_total,V_total, S0, nb_T, 
                                 tau, alpha, eta, epsilon, sigma, type_func)


KeyboardInterrupt: 

In [ ]:
a, b = 1e-04, 1e-05
risks = np.linspace(a,b, 300)
type_func = 'VWAP'
x = []
y = []
q_trajectories =[]
for r in risks:
    temp = efficient_frontier(X_total,V_total, S0, r, nb_T, 
                                 tau, alpha, eta, epsilon,sigma, type_func)
    q_trajectories.append(temp[0])
    x.append(temp[2])
    y.append(temp[1])
    
plt.figure(figsize=(12,6))
plt.scatter(x, y)
plt.grid(True)
plt.xlabel('Variance V(x)')
plt.ylabel('Expectation E(x)')
plt.title('Efficient frontier for '+ type_func)
plt.show()